In [1]:
import pandas as pd

df = pd.read_csv('resultados_comparacao.csv')
df

,instancia,guloso,vnd
0,n3m10A.txt,23940,10194
1,n3m10B.txt,5559,1561
2,n3m10C.txt,10301,2150
3,n3m10D.txt,5124,322
4,n3m10E.txt,6920,3343
5,n3m20A.txt,88820,38574
6,n3m20B.txt,66010,23918
7,n3m20C.txt,33006,8908
8,n3m20D.txt,16477,4465
9,n3m20E.txt,33326,4537


In [3]:
df_gap2 = pd.read_csv("resultados/gap_25 iter.csv")
display(df_gap2)

,Instancia,Otimo,Tag otimo,Solucao (HC),Tempo (HC),Gap (HC),Solucao (VND),Tempo (VND),Gap (VND),Solucao (GRASP),Tempo (GRASP),Gap (GRASP)
0,n3m10A.txt,7483,opt,23940,0.0005 segundos,219.93,7483,0.0334 segundos,0.00,7483,1.9986 segundos,0.00
1,n3m10B.txt,1277,opt,5559,0.0004 segundos,335.32,1277,0.0290 segundos,0.00,1277,1.1370 segundos,0.00
2,n3m10C.txt,2088,opt,10301,0.0004 segundos,393.34,2150,0.0452 segundos,2.97,2088,1.4924 segundos,0.00
3,n3m10D.txt,322,opt,5124,0.0004 segundos,1491.30,322,0.0422 segundos,0.00,322,2.0337 segundos,0.00
4,n3m10E.txt,3343,opt,6920,0.0004 segundos,107.00,3452,0.0287 segundos,3.26,3343,1.0294 segundos,0.00
5,n3m20A.txt,8280,LB,88820,0.0006 segundos,972.71,38393,0.6174 segundos,363.68,31357,28.3967 segundos,278.71
6,n3m20B.txt,1820,LB,66010,0.0005 segundos,3526.92,23354,0.2878 segundos,1183.19,16719,23.4327 segundos,818.63
7,n3m20C.txt,855,LB,33006,0.0005 segundos,3760.35,8293,0.2745 segundos,869.94,6462,18.7469 segundos,655.79
8,n3m20D.txt,4357,opt,16477,0.0005 segundos,278.17,4785,0.2982 segundos,9.82,4357,13.3037 segundos,0.00
9,n3m20E.txt,3798,opt,33326,0.0005 segundos,777.46,3827,0.2988 segundos,0.76,3798,26.5780 segundos,0.00
